Maxwell solver for Pec using direct formulation
=============================
**keys**: Maxwell double layer potential, PEC scattering, MoM, Neumann trace

ASSUME: 

    return make_unique<GenericIntegralOperator<MaxwellDLKernel<3>>>(space, space,                                                            
                                                                    make_shared<T_DifferentialOperator<DiffOpRotatedTrace>>(), 
                                                                    make_shared<T_DifferentialOperator<DiffOpRotatedTrace>>(), 
                                                                    MaxwellDLKernel<3>(kappa), param);


In [ ]:
from netgen.occ import *
import netgen.meshing as meshing
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG

We consider a perfect conductor $\Omega \subset \mathbb R^3$ and a plane wave $\boldsymbol E_{\mathrm{inc}} $ with tangential trace $\gamma_R \boldsymbol E_{\mathrm{inc}} = -\boldsymbol m \in \boldsymbol H^{-\frac12}\left( \mathrm{curl}_\Gamma, \Gamma\right)\,.$ The incoming wave thus induces a scattered electric field $\boldsymbol E$ which propagates into $\Omega^c$. The scattered electric field solves the following Dirichlet boundary value problem: 

$$ \left\{ \begin{array}{rcl l} \mathbf{curl} \, \mathbf{curl}\, \boldsymbol E - \kappa^2 \, \boldsymbol E &=& \boldsymbol 0, \quad &\textnormal{in } \Omega^c \subset \mathbb R^3\,,\\ \gamma_R \,\boldsymbol E &=& \boldsymbol m_R, \quad & \textnormal{on }\Gamma \\ \left\| \mathbf{curl} \, \boldsymbol E( x) - i\,\omega\,\epsilon \, \boldsymbol E( x)\right\| &=& \mathcal O\left( \displaystyle \frac{1}{\| x\|^2}\right), &\textnormal{for} \; \|x\| \to \infty\,.\end{array} \right. $$ 

and a possible representation reads

$$ \boldsymbol E(x) = \underbrace{\int\limits_\Gamma \displaystyle{ \frac{1}{4\,\pi} \, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \boldsymbol j(y)\, \mathrm{d}\sigma_y + \frac{1}{\kappa^2} \nabla \int\limits_\Gamma \displaystyle{ \frac{1}{4\,\pi}\, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \mathrm{div}_\Gamma \boldsymbol j(y)\, \mathrm{d}\sigma_y }_{\displaystyle{\mathrm{SL}(\boldsymbol j)} } + \underbrace{ \nabla \times \int\limits_\Gamma \displaystyle{ \frac{1}{4\,\pi} \, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \boldsymbol m_D(y)\, \mathrm{d}\sigma_y }_{\displaystyle{ \mathrm{DL} ( \boldsymbol{m}_D) } } \,.$$

The trace $\boldsymbol j$ is the Neumann trace and $\boldsymbol{ m}_D = \boldsymbol n\times \boldsymbol m_R$ denotes the rotated Dirichlet trace of $\boldsymbol E$. We carefully apply the tangential trace $\gamma_R$ and thus obtain a boundary integral equation for unknown Neumann data $\boldsymbol j$. The bounary integral equation is solved by the boundary element method, i.e. the numerical solution of the variational formulation 

$$  \forall \, \boldsymbol v\in H^{-\frac12}(\mathrm{div}_\Gamma, \Gamma): \quad \left\langle \mathrm{SL} (\boldsymbol j), \boldsymbol v \right\rangle_{-\frac12} = \left\langle \boldsymbol{m}_D, \boldsymbol v\right\rangle_{-\frac12}  + \left\langle \mathrm{DL}( \boldsymbol{m}_D), \boldsymbol v\right\rangle_{-\frac12} \,. $$ 

In the enineering community, the approximation scheme is also known as **method of moments** (MoM). 

Define the geometry of the perfect conductor $\Omega$ and create a mesh:

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=1, perfstepsend=meshing.MeshingStep.MESHSURFACE)).Curve(2)

Next, we create finite element spaces for $\boldsymbol H^{-\frac12}(\mathrm{curl}_\Gamma, \Gamma)$ and rotate them later to obtain $\boldsymbol H^{-\frac12}(\mathrm{div}_\Gamma, \Gamma)$ conforming shape functions as test and trial space. 

In [ ]:
fesHCurl = HCurl(mesh, order=3, complex=True)
fesHDiv = HDivSurface(mesh, order=3, complex=True)
uHCurl,vHCurl = fesHCurl.TnT() # H(curl_Gamma) conform spaces; trial and test
uHDiv,vHDiv = fesHDiv.TnT() # H(div_Gamma) conform spaces; trial and test

print ("ndof HCurl = ", fesHCurl.ndof)

Define the incoming plane wave and compute the given boundary data $\boldsymbol m = -\gamma_R \boldsymbol E_{\mathrm{inc}}$ : 

In [ ]:
eps0 = 8.854e-12 
mu0 = 4*pi*1e-7
omega = 1.5e9
kappa = omega*sqrt(eps0*mu0)
print("kappa = ", kappa)

E_inc = CF((1,0,0))*exp( -1j * kappa * z )

n = specialcf.normal(3)
mR = GridFunction(fesHCurl) 
mR.Set( - E_inc, definedon=mesh.Boundaries(".*"), dual=True) # Hcurl
#Draw (Norm(mR), mesh, draw_vol=False, order=2);

The discretisation of the above variational formulation leads to a system of linear equations, ie
$$ \mathrm{V} \, \mathrm j = \left( \frac12 \,\mathrm{M} +\mathrm{K} \right)\, \mathrm{m_R}\,,$$
where 
* $\mathrm V$ is the Maxwell single layer operator. $V$ is a regular, symmetric matrix.
* $\mathrm{M}$ is the mass matrix.
* $\mathrm K$ is the Maxwell double layer operator. $K$ is quadratic. 

In [ ]:
with TaskManager(): 
    V = MaxwellSingleLayerPotentialOperatorNew(fesHDiv, kappa, 
                                            intorder=10, leafsize=40, eta=3., eps=1e-4, method="aca", testhmatrix=False)
    K = MaxwellDoubleLayerPotentialOperatorNew(fesHCurl, fesHDiv, kappa, 
                                            intorder=12, leafsize=40, eta=3., eps=1e-6, method="aca", testhmatrix=False)

In [ ]:
j = GridFunction(fesHDiv) 
pre = BilinearForm( uHDiv.Trace() * vHDiv.Trace() *ds).Assemble().mat.Inverse(freedofs=fesHCurl.FreeDofs()) 
with TaskManager(): 
    M = BilinearForm(uHCurl.Trace() * vHDiv.Trace() * ds(bonus_intorder=3)).Assemble() # <Hcurl, Hdiv>  
    rhs = ( (0.5 * M.mat + K.mat) * mR.vec).Evaluate()
    CG(mat = V.mat, pre=pre, rhs = rhs, sol=j.vec, tol=1e-8, maxsteps=500, initialize=False, printrates=False)

In [ ]:
## alternatively work with rotated trace mD
#mD = GridFunction(fesHCurl) 
#mD.Set( - Cross(n,E_inc), definedon=mesh.Boundaries(".*"), dual=True) # Hdiv
#j = GridFunction(fesHCurl) 
#pre = BilinearForm( uHCurl.Trace() * vHCurl.Trace() *ds).Assemble().mat.Inverse(freedofs=fesHCurl.FreeDofs()) 
#with TaskManager(): 
#    M = BilinearForm(uHCurl.Trace() * vHCurl.Trace() * ds(bonus_intorder=3)).Assemble() # <Hcurl, Hdiv>  
#    rhs = ( 0.5 * M.mat * mD.vec + K.mat * mR.vec).Evaluate() 
#    CG(mat = V.mat, pre=pre, rhs = rhs, sol=j.vec, tol=1e-8, maxsteps=500, initialize=False, printrates=False)

In [ ]:
Draw(j, mesh, draw_vol=False, order=3, animate_complex=True);

In [ ]:
bis daher hab ich auf HDivSurface umgestellt ....

**Check the numerical result for $j$ with EFIE**

The density $\boldsymbol j$ is related to the solution $\boldsymbol j_{\mathrm{efie}}$ of the EFIE (indirect formulation), i.e., 

$$  \boldsymbol j = \boldsymbol j_{\mathrm{efie}} - \boldsymbol j_{\mathrm{inc}}\,.$$ 

Thus, given $\boldsymbol j_{\mathrm{efie}}$, there is an alternative to compute $\boldsymbol j$. We use this alternative to check the direct formulation.

In [ ]:
# solve EFIE (Neumann trace of total field)
j_efie = GridFunction(fesHCurl) 
rhs_efie = LinearForm( mR * Cross( n, vHCurl.Trace() ) *ds(bonus_intorder=3)).Assemble() 
CG(mat = V.mat, pre=pre, rhs = rhs_efie.vec, sol=j_efie.vec, tol=1e-8, maxsteps=500, initialize=False, printrates=False);

In [ ]:
# get the resulting Neumann trace of the scattered field
curlE_inc = CF( (0,-1j*kappa,0) ) *exp( -1j * kappa * z ) 
j_inc = GridFunction(fesHCurl) 
j_inc.Set( curlE_inc, definedon=mesh.Boundaries(".*")) 
j_test = GridFunction(fesHCurl) 
j_test.Set (j_efie-j_inc, definedon=mesh.Boundaries(".*")) 

In [ ]:
Draw(Norm(j), mesh, draw_vol=False, order=2);
Draw(Norm(j_test), mesh, draw_vol=False, order=2, animate_complex=True);
print(Norm(j.vec))
print(Norm(j_test.vec))

In [ ]:
Draw(j.real, mesh, draw_vol=False, order=2);
Draw(j_test.real, mesh, draw_vol=False, order=2);

In [ ]:
Draw(j.imag, mesh, draw_vol=False, order=2);
Draw(j_test.imag, mesh, draw_vol=False, order=2);

For details on convergence studies for HOBEM for PEC scattering look [here](https://publikationen.sulb.uni-saarland.de/bitstream/20.500.11880/26312/1/thesis_weggler_final_6.1.12.pdf).

In [ ]:
fesHCurl = HCurl(mesh, order=3, complex=True)
fesHDiv = HDivSurface(mesh, order=3, complex=True)

K = MaxwellDoubleLayerPotentialOperator(fesHCurl, fesHDiv, kappa, 
                                        intorder=12, leafsize=40, eta=3., eps=1e-6, method="aca", testhmatrix=False)

In [ ]:
print (K.mat)